In [1]:
import pandas as pd
from jinja2 import Environment, FileSystemLoader
import requests
from io import StringIO
import os

In [2]:
# PARAMETERS
DISTANCE_RANGE = 8

In [3]:
def get_distance(row_nr, seat_nr):
    distance = seat_nr
    if row_nr not in [1,2]:
        distance = distance + 12
    if row_nr > 3:
        distance = distance + 30
    if row_nr in [4,8,9,10,11,12]:
        distance = distance + 6
    if row_nr > 8:
        distance = distance + 24
    return distance

In [4]:
def find_all_options(location, number_of_seats):
    possible = []
    for row_nr, row in enumerate(location):
        for seat_nr in range(len(row)-number_of_seats+1):
            last_nr = seat_nr + number_of_seats-1
            if all([True if _nr is None else False for _nr in row[seat_nr:last_nr+1]]): # every seat in window must be free
                possible.append((row_nr, seat_nr, last_nr, get_distance(row_nr, last_nr)))
                break
    return possible

In [5]:
def find_options(location, number_of_seats,row_wishes):
    options = find_all_options(location, number_of_seats)
    if len(options) > 0:
        if len(row_wishes) > 0:
            return [option for option in options if option[0] in row_wishes]
        else:
            min_distance = min([option[-1] for option in options])
            return [option for option in options if option[-1] <= min_distance + DISTANCE_RANGE]
    else:
        return []

In [6]:
def validate_options(location, options):
    valid_options = []
    upper_rows = [9,10,11,12]
    for option in options:
        valid = True
        row_nr, seat_nr, last_nr, _ = option
        row = location[row_nr]
        free_in_row = sum([1 if seat is None else 0 for seat in row])
        number_of_seats = last_nr - seat_nr + 1
        if row_nr < 4 and number_of_seats == free_in_row - 1: # we don't want to let one single seat free in a row
            valid = False
        
        if number_of_seats%2 == 0 and seat_nr%2 == 1: # not a diagonal positioning of i.e. pairs
            valid = False
        
        if number_of_seats == free_in_row:
            valid = True
    
        if valid:
            valid_options.append(option)
    return valid_options
        

In [7]:
def get_row_wishes(reservations, res_ids):
    wish_list = ",".join([res[3] for res in reservations if res[0] in res_ids and res[3] != ""])
    wish_list = wish_list.split(",") if len(wish_list) > 0 else []
    return [int(float(wish)) for wish in wish_list]
    

In [8]:
def allocate_seat(location, reservation_seats, row_wishes):
    number_of_seats = len(reservation_seats)
    options = find_options(location, number_of_seats, row_wishes)
    valid_options = validate_options(location, options)
    if len(valid_options) > 0:
        sorted_options = sorted(valid_options, key=lambda o: o[3])
        row_nr, seat_nr, last_nr, _ = sorted_options[0]
        
        for offset in range(number_of_seats):
            location[row_nr][seat_nr+offset] = reservation_seats[offset]
        
        return True
    return False
    

In [9]:
def get_all_reservations():
    user = os.getenv("SEATY_API_USER")
    pwd = os.getenv("SEATY_API_PWD")
    # Define the API URL and credentials
    url = os.getenv("SEATY_API_URL")
    auth = (user, pwd)
    
    # Send a GET request to the API with Basic Authentication
    response = requests.get(url, auth=auth)
    
    csv_data = StringIO(response.text)
    df = pd.read_csv(csv_data, delimiter=';')
    
    data_types_dict = {
        'seats': int,
        'group': int,
        'event_id': int,
        'preferred_row': str,
        'comment': str,
        'internal_comment': str
    }
    default_values = {'seats': 0, 'group': -1, 'preferred_row': "", 'comment': "", 'internal_comment': ""}
    
    reservations = df.fillna(value=default_values).astype(data_types_dict)
    reservations = reservations.sort_values(['prio', 'code'], ascending=[False, True])
    return reservations

In [10]:
def create_allocation(reservations):

    location=[]
    for _ in range(4):
        location.append([None for _ in range(1,25)])

    for _ in range(5):
        location.append([None for _ in range(1,20)])

    for _ in range(4):
        location.append([None for _ in range(1,5)])

    reservations = reservations[['code', 'seats', 'group','preferred_row', 'name', 'comment', 'internal_comment']]
    reservations = reservations[reservations['seats']>0]
    reservations_list = reservations.values.tolist()
    res_idx, cards_idx, group_idx, wish_idx, name_idx = 0, 1, 2, 3, 4

    # group by groups
    groups = []
    current_group = 0
    current_group_seats = []
    for res in reservations_list:
        if res[group_idx] != current_group or res[group_idx] == -1:
            if len(current_group_seats) > 0:
                groups.append(current_group_seats)
            current_group_seats = []
            current_group = res[group_idx]

        for _ in range(res[cards_idx]):
            current_group_seats.append(res[res_idx])

    if len(current_group_seats) > 0:
        groups.append(current_group_seats)

    idx = 0
    assigned_reservations = []
    while idx < len(groups):
        group_seats = groups[idx]
        if idx not in assigned_reservations:
            row_wishes = get_row_wishes(reservations_list, set(group_seats))
            allocated = allocate_seat(location, group_seats, row_wishes)
            if allocated:
                assigned_reservations.append(idx)
                idx = -1
        idx += 1
    
    not_allocated_idx = set([i for i in range(len(groups))]) - set(assigned_reservations)
    not_allocated = [groups[i] for i in not_allocated_idx]

    print(f"Allocation finished for event {event}")
    print(f"all groups: {len(groups)}, assigned {len(assigned_reservations)}")
    print(sum(res[cards_idx] for res in reservations_list))
    print(sum([0 if res is None else 1 for row in location for res in row]))
    print("not allocated groups", not_allocated)
    print("------------------------------------------------------")
    
    # make reservation overview html
    environment = Environment(loader=FileSystemLoader("templates/"))
    template_file = environment.get_template("location_template.html")
    rendered_file = f"./generated/event_{event}_location.html"

    context = {
        "block_1":location[:4],
        "block_2":location[4:9],
        "block_3":location[9:],
        "res_to_name": pd.Series(reservations['name'].values,index=reservations['code']).to_dict()
    }

    with open(rendered_file, mode="w", encoding="utf-8") as results:
        results.write(template_file.render(context))
        
    # make reservation cards
    reservation_file = environment.get_template("reservations_template.html")
    reservation_out_file = f"./generated/event_{event}_cards.html"
    context_reservations = {
        "reservations": reservations.to_dict('records')
    }
          
    with open(reservation_out_file, mode="w", encoding="utf-8") as results:
        results.write(reservation_file.render(context_reservations))


In [11]:
def create_event_allocation(reservations, event):
    event_reservations = reservations[reservations['event_id']==event]
    create_allocation(event_reservations)

In [12]:
reservations = get_all_reservations()
for event in [1,2,3,4,5,6]:
    create_event_allocation(reservations, event)

Allocation finished for event 1
all groups: 18, assigned 17
125
102
not allocated groups [[1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1002, 1008, 1008, 1008]]
------------------------------------------------------
Allocation finished for event 2
all groups: 20, assigned 20
94
94
not allocated groups []
------------------------------------------------------
Allocation finished for event 3
all groups: 14, assigned 14
44
44
not allocated groups []
------------------------------------------------------
Allocation finished for event 4
all groups: 19, assigned 9
86
54
not allocated groups [[4008, 4008, 4008, 4008], [4009, 4009, 4009, 4009], [4011, 4011, 4019, 4019], [4014, 4014], [4015, 4015], [4016, 4016], [4017, 4017, 4017, 4017], [4018, 4018, 4018, 4018], [4020, 4020], [4021, 4021, 4021, 4021]]
------------------------------------------------------
Allocation finished for event 5
all groups: 14, assigned 14
56
56
not a